# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [26]:
# Dependencies and Setup
import hvplot.pandas
import hvplot
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [27]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wipperfurth,51.1167,7.4000,10.68,96,100,4.53,DE,1680357351
1,1,launceston,-41.4500,147.1667,8.97,80,83,2.06,AU,1680357352
2,2,hermanus,-34.4187,19.2345,24.89,52,0,7.88,ZA,1680357352
3,3,punta arenas,-53.1500,-70.9167,7.06,70,75,4.63,CL,1680357352
4,4,koslan,63.4564,48.8989,0.05,99,100,1.67,RU,1680357352


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
city_temp_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    alpha = 0.5,
    color = "City"    
)


# Display the map
city_temp_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df.loc[((city_data_df['Max Temp']>21)&
                              (city_data_df['Max Temp']<27))&
                              (city_data_df['Wind Speed']<4.5)&
                              (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
ideal_city.dropna()

# Display sample data
ideal_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
62,62,port alfred,-33.5906,26.8910,23.43,70,0,2.24,ZA,1680357375
107,107,port elizabeth,-33.9180,25.5701,26.17,33,0,4.12,ZA,1680357403
331,331,jalu,29.0331,21.5482,25.90,13,0,2.15,LY,1680357577
332,332,marsa matruh,31.3525,27.2453,23.11,31,0,4.12,EG,1680357577
374,374,acapulco,16.8634,-99.8901,26.29,56,0,1.15,MX,1680357308
381,381,lazaro cardenas,17.9583,-102.2000,24.37,65,0,1.82,MX,1680357583
428,428,russell,32.3502,-85.1999,21.89,83,0,2.57,US,1680357588
463,463,les cayes,18.2000,-73.7500,26.87,66,0,3.40,HT,1680357592
484,484,houston,29.7633,-95.3633,23.87,93,0,3.09,US,1680356796
512,512,romitan,39.9343,64.3814,26.97,22,0,3.09,UZ,1680357598


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
62,port alfred,ZA,-33.5906,26.8910,70,
107,port elizabeth,ZA,-33.9180,25.5701,33,
331,jalu,LY,29.0331,21.5482,13,
332,marsa matruh,EG,31.3525,27.2453,31,
374,acapulco,MX,16.8634,-99.8901,56,
381,lazaro cardenas,MX,17.9583,-102.2000,65,
428,russell,US,32.3502,-85.1999,83,
463,les cayes,HT,18.2000,-73.7500,66,
484,houston,US,29.7633,-95.3633,93,
512,romitan,UZ,39.9343,64.3814,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
radius = 10000
limit = 50
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city = hotel_df.loc[index, 'City']
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port alfred - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
jalu - nearest hotel: فندق جالو السياحي
marsa matruh - nearest hotel: صقور
acapulco - nearest hotel: Hotel del Valle
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
russell - nearest hotel: No hotel found
les cayes - nearest hotel: No hotel found
houston - nearest hotel: Hotel ICON
romitan - nearest hotel: Темур hotel
kashan - nearest hotel: Parsi


,City,Country,Lat,Lng,Humidity,Hotel Name
62,port alfred,ZA,-33.5906,26.8910,70,No hotel found
107,port elizabeth,ZA,-33.9180,25.5701,33,Waterford Hotel
331,jalu,LY,29.0331,21.5482,13,فندق جالو السياحي
332,marsa matruh,EG,31.3525,27.2453,31,صقور
374,acapulco,MX,16.8634,-99.8901,56,Hotel del Valle
381,lazaro cardenas,MX,17.9583,-102.2000,65,Hotel Sol del Pacífico
428,russell,US,32.3502,-85.1999,83,No hotel found
463,les cayes,HT,18.2000,-73.7500,66,No hotel found
484,houston,US,29.7633,-95.3633,93,Hotel ICON
512,romitan,UZ,39.9343,64.3814,22,Темур hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = 'OSM',
    hover_cols = ['Hotel Name', 'Country'],
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.5,
    alpha = 0.5,
    color = 'City'   
)


# Display the map
city_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)